# Assignment 2
**COMP 511: Network Science**

**Due on February 6th 2026**

In [5]:
import pandas as pd
import networkx as nx
from collections import Counter, defaultdict
import gdown
import tarfile
import os
import re
import numpy as np

## 1. Centrality Measures [20%]

Select 3 (or more) centrality measures and find the top 5 most important nodes in the [Enron dataset](https://www.cs.cornell.edu/~arb/data/pvc-email-Enron/). Who are the top ranked people?

*   aggregate all emails sent at different times into a static snapshot with an edge weight showing how many emails in total have been send from one node to the other

In [6]:
# google drive file ID
file_id = "1hbpdnqvTRqPZ4MvzxeKLLkifdXZhtQsn"

output_file = "email-Enron.tar.gz"

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file, quiet=False)

# Extract the .tar.gz file
try:
    with tarfile.open(output_file, "r:gz") as tar:
        tar.extractall()  # extract into "email-Enron" folder
        print("Extraction completed successfully.")
except tarfile.ReadError as e:
    print("Error reading the tar.gz file:", e)

Downloading...
From: https://drive.google.com/uc?id=1hbpdnqvTRqPZ4MvzxeKLLkifdXZhtQsn
To: /Users/barthelemyderivieres/Desktop/McGill/U4 sem 2/COMP511/A1/assignments/A2/email-Enron.tar.gz
100%|██████████| 1.66M/1.66M [00:00<00:00, 11.9MB/s]

Extraction completed successfully.


In [7]:
data_folder = 'email-Enron'
addresses_path = f"{data_folder}/addresses-email-Enron.txt"
core_path = f"{data_folder}/core-email-Enron.txt"
email_path = f"{data_folder}/email-Enron.txt"

# read the addresses
addresses_df = pd.read_csv(
    addresses_path,
    sep=r'\s+',
    header=None,
    names=['ID', 'email_address'],
    on_bad_lines='skip'
)
addresses_df.set_index('ID', inplace=True)

# read core ids
core_df = pd.read_csv(
    core_path,
    sep=r'\s+',
    header=None,
    names=['ID']
)

# use set for efficiency
core_ids = set(core_df['ID'].tolist())

# read email edges
edges_df = pd.read_csv(
    email_path,
    sep=r'\s+',
    header=None,
    names=['sender', 'recipient', 'timestamp']
)

# we need to aggregate the edge weights to get a static snapshot that we can use
agg_edges = edges_df.groupby(['sender', 'recipient']).size().reset_index(name='weight')

# now lets make the static graph
G_static = nx.DiGraph()

# add weights between nodes based on agg_edges
for row in agg_edges.itertuples(index=False):
    G_static.add_edge(row.sender, row.recipient, weight=row.weight)

# add the email as a node attribute so we can see important names later
for node in G_static.nodes():
  try:
    G_static.nodes[node]['email_address'] = addresses_df.loc[node, 'email_address']
  except KeyError:
    pass

In [8]:
# compute centrality measures on the full directed graph
deg_cent = nx.degree_centrality(G_static)
eig_cent = nx.eigenvector_centrality(G_static, max_iter=1000)
betw_cent = nx.betweenness_centrality(G_static)

# HITS: returns (hubs, authorities) dicts
hits_hubs, hits_auths = nx.hits(G_static, max_iter=1000)

# helper to look up email from node id
def node_to_email(nid):
    if nid in addresses_df.index:
        return addresses_df.loc[nid, 'email_address']
    return f"Unknown({nid})"

# display top 5 for each measure
measures = {
    'Degree': deg_cent,
    'Eigenvector': eig_cent,
    'Betweenness': betw_cent,
    'HITS Hub': hits_hubs,
    'HITS Authority': hits_auths,
}

for name, scores in measures.items():
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"\nTop 5 by {name} Centrality:")
    for i, (nid, val) in enumerate(ranked, 1):
        print(f"  {i}. {node_to_email(nid)} — {val:.5f}")

KeyboardInterrupt: 

## 2. Graph Clustering [50%]

Select two (or more) community detection or graph clustering algorithms, apply them on the following real world datasets and evaluate their performances:

*   -- real-classic: [strike, karate, polblog, polbooks, football](http://www.reirab.com/Teaching/NS20/classic.tar.gz)
*   -- real-node-label: [citeseer, cora, pubmed](https://github.com/tkipf/gcn/tree/master/gcn/data)

Here, the goal is to use the classification labels as clustering labels, and see how well we can find those labels without using the feature vectors.

In [ ]:
download_folder = "classic_graph_data/"
os.makedirs(download_folder, exist_ok=True)

file_ids = {
    "polbooks.gml": "1SzhpzmpEO9NSjEabjoZP16NzYN3s6RBn",
    "polbooks.txt": "1yEzPMsGPUwrGepoxN4hlX3a54CvAPLj-",
    "polblogs.gml": "10ptbBiMgm541aAr3Rd4svBFia6qjJOGV",
    "polblogs.txt": "1un9r36q4qtuz9Kjv7MqVifMp0pH45Es2",
    "karate.gml": "1zvpOcA_74qO0pp9o36xg2rdwx9pTkgW_",
    "karate.txt": "1-i66II9DThEJruAWoslgb2Cc88R17jBK",
    "strike.gml": "137lB6IHqkRzHYOeYBn-0n4pNIYN3tuvr",
    "football.gml": "1ooEj5eBI7UM2ZtfexmaTxIFJt6qn43Uy",
    "football.txt": "126uHBR218fWCBRJjORSmaaxoHUGfTbPp",
}

for file_name, file_id in file_ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    output_path = os.path.join(download_folder, file_name)
    gdown.download(url, output_path, quiet=False)
    print(f"Downloaded: {file_name}")

download_folder = "labeled_graph_data/"
os.makedirs(download_folder, exist_ok=True)

file_ids = {
    "citeseer_node_labels": "1u51oSH8O46i3WnG_RqHrxcZnTURod20l",
    "citeseer_edges": "1IceUHrm8fk51Fb_gLHmXg0kETcCZa2bw",
    "cora_cites": "1AC_LOmycApD7_MtahW6WeaxSOzhPPp9J",
    "cora_content": "1mQ9dWt_8jJr0pJFiyTsKS2hGEE85YPZQ",
    "pubmed_node_labels": "1HUlVeZCjiO4GEQrQgH282VPC835mfPaO",
    "pubmed_edges": "1lss1_snvRj8sx4Aoo-vS2bJt62j_3YPt"
}

for file_name, file_id in file_ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    output_path = os.path.join(download_folder, file_name)
    gdown.download(url, output_path, quiet=False)
    print(f"Downloaded: {file_name}")

Downloading...
From: https://drive.google.com/uc?id=1SzhpzmpEO9NSjEabjoZP16NzYN3s6RBn
To: /Users/barthelemyderivieres/Desktop/McGill/U4 sem 2/COMP511/A1/assignments/A2/classic_graph_data/polbooks.gml
100%|██████████| 26.3k/26.3k [00:00<00:00, 18.0MB/s]


Downloaded: polbooks.gml


Downloading...
From: https://drive.google.com/uc?id=1yEzPMsGPUwrGepoxN4hlX3a54CvAPLj-
To: /Users/barthelemyderivieres/Desktop/McGill/U4 sem 2/COMP511/A1/assignments/A2/classic_graph_data/polbooks.txt
100%|██████████| 638/638 [00:00<00:00, 1.31MB/s]


Downloaded: polbooks.txt


Downloading...
From: https://drive.google.com/uc?id=10ptbBiMgm541aAr3Rd4svBFia6qjJOGV
To: /Users/barthelemyderivieres/Desktop/McGill/U4 sem 2/COMP511/A1/assignments/A2/classic_graph_data/polblogs.gml
100%|██████████| 978k/978k [00:00<00:00, 6.89MB/s]


...

Downloaded: cora_content


Downloading...
From: https://drive.google.com/uc?id=1HUlVeZCjiO4GEQrQgH282VPC835mfPaO
To: /Users/barthelemyderivieres/Desktop/McGill/U4 sem 2/COMP511/A1/assignments/A2/labeled_graph_data/pubmed_node_labels
100%|██████████| 208k/208k [00:00<00:00, 2.90MB/s]


Downloaded: pubmed_node_labels


Downloading...
From: https://drive.google.com/uc?id=1lss1_snvRj8sx4Aoo-vS2bJt62j_3YPt
To: /Users/barthelemyderivieres/Desktop/McGill/U4 sem 2/COMP511/A1/assignments/A2/labeled_graph_data/pubmed_edges
100%|██████████| 761k/761k [00:00<00:00, 5.00MB/s]

Downloaded: pubmed_edges


In [ ]:
def clean_gml_file(input_path, output_path, gml_filename):

    with open(input_path, 'r') as f:
        lines = f.readlines()

    seen_edges = set()
    new_lines = []
    inside_edge_block = False
    buffer_block = []
    edge_counter = 0

    for line in lines:
        stripped_line = line.rstrip("\n\r")

        if stripped_line.strip().startswith("edge ["):
            inside_edge_block = True
            buffer_block = [line]  # start buffering this edge block
            continue

        if inside_edge_block:
            buffer_block.append(line)

            if stripped_line.strip().startswith("]"):
                inside_edge_block = False
                edge_counter += 1

                block_text = "".join(buffer_block)
                match_source = re.search(r'\bsource\s+(\d+)', block_text)
                match_target = re.search(r'\btarget\s+(\d+)', block_text)

                if match_source and match_target:
                    source = int(match_source.group(1))
                    target = int(match_target.group(1))

                    if source != target:
                        edge_tuple = tuple(sorted((source, target)))

                        if edge_tuple not in seen_edges:
                            seen_edges.add(edge_tuple)
                            new_lines.extend(buffer_block)
                buffer_block = []

        else:
            new_lines.append(line)
    with open(output_path, 'w') as out:
        out.writelines(new_lines)

classic_graph_folder = "classic_graph_data/"
classic_cleaned_folder = "classic_graph_data_cleaned/"
os.makedirs(classic_cleaned_folder, exist_ok=True)

classic_gml_files = [f for f in os.listdir(classic_graph_folder) if f.endswith(".gml")]
classic_graphs = {}

for gml_file in classic_gml_files:
    input_path = os.path.join(classic_graph_folder, gml_file)
    output_path = os.path.join(classic_cleaned_folder, gml_file)
    clean_gml_file(input_path, output_path, gml_file)
    try:
        G = nx.read_gml(output_path)
        if isinstance(G, nx.MultiDiGraph) or isinstance(G, nx.DiGraph):
            G = nx.Graph(G)
        G.remove_edges_from(nx.selfloop_edges(G))
        for node, data in G.nodes(data=True):
            if 'value' in data:
                G.nodes[node]['label'] = data['value']

        classic_graphs[gml_file] = G
        print(f"[LOADED] {gml_file} -> {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")
    except nx.NetworkXError as e:
        print(f"[ERROR] Could not load {gml_file}: {e}")

Loading 'classic' graphs. Note: These are often referred to as unlabeled, but most (except strike) do have ground truth labels available.
[CLEAN] Removed duplicates. Cleaned file saved to: classic_graph_data_cleaned/polblogs.gml
[LOADED] polblogs.gml -> 1490 nodes, 16715 edges.
[CLEAN] Removed duplicates. Cleaned file saved to: classic_graph_data_cleaned/strike.gml
[LOADED] strike.gml -> 24 nodes, 38 edges.
[CLEAN] Removed duplicates. Cleaned file saved to: classic_graph_data_cleaned/karate.gml
[LOADED] karate.gml -> 34 nodes, 78 edges.
[CLEAN] Removed duplicates. Cleaned file saved to: classic_graph_data_cleaned/football.gml
[LOADED] football.gml -> 115 nodes, 613 edges.
[CLEAN] Removed duplicates. Cleaned file saved to: classic_graph_data_cleaned/polbooks.gml
[LOADED] polbooks.gml -> 105 nodes, 441 edges.


In [ ]:
graph_folder = "labeled_graph_data"
edges_file = os.path.join(graph_folder, "citeseer_edges")
labels_file = os.path.join(graph_folder, "citeseer_node_labels")

def load_citeseer_graph(edges_file, labels_file):
    G = nx.Graph()

    with open(edges_file, "r") as f:
        for line in f:
            src, dst, *_ = line.strip().split(",")
            G.add_edge(int(src), int(dst))

    labels = {}
    with open(labels_file, "r") as f:
        for line in f:
            node, label = line.strip().split(",")
            node, label = int(node), int(label)
            G.nodes[node]["label"] = label
            labels[node] = label

    return G, labels

G_citeseer, citeseer_labels = load_citeseer_graph(edges_file, labels_file)
print(f"Loaded CiteSeer graph with {G_citeseer.number_of_nodes()} nodes and {G_citeseer.number_of_edges()} edges.")

edges_file = os.path.join(graph_folder, "cora_cites")
content_file = os.path.join(graph_folder, "cora_content")

def load_cora_graph(edges_file, content_file):
    G = nx.Graph()
    node_features = {}
    node_labels = {}

    with open(content_file, "r") as f:
        for line in f:
            parts = line.strip().split("\t") 
            node_id = int(parts[0])
            features = np.array([int(x) for x in parts[1:-1]]) 
            label = parts[-1]  

            G.add_node(node_id, features=features, label=label)
            node_features[node_id] = features
            node_labels[node_id] = label

    with open(edges_file, "r") as f:
        for line in f:
            src, dst = map(int, line.strip().split("\t"))
            G.add_edge(src, dst)

    return G, node_features, node_labels

G_cora, cora_features, cora_labels = load_cora_graph(edges_file, content_file)
print(f"Loaded Cora graph with {G_cora.number_of_nodes()} nodes and {G_cora.number_of_edges()} edges.")


edges_file = os.path.join(graph_folder, "pubmed_edges")
labels_file = os.path.join(graph_folder, "pubmed_node_labels")

def load_pubmed_graph(edges_file, labels_file):
    G = nx.Graph()

    with open(edges_file, "r") as f:
        for line in f:
            src, dst = map(int, line.strip().split(","))
            G.add_edge(src, dst)

    labels = {}
    with open(labels_file, "r") as f:
        for line in f:
            node, label = line.strip().split(",")
            node, label = int(node), int(label)
            G.nodes[node]["label"] = label
            labels[node] = label

    return G, labels

G_pubmed, pubmed_labels = load_pubmed_graph(edges_file, labels_file)
print(f"Loaded PubMed graph with {G_pubmed.number_of_nodes()} nodes and {G_pubmed.number_of_edges()} edges.")

Loaded CiteSeer graph with 3264 nodes and 4536 edges.
Loaded Cora graph with 2708 nodes and 5278 edges.
Loaded PubMed graph with 19717 nodes and 44327 edges.


### 2(a) Algorithmic Complexity [10%]
Derive and report the complexity of the chosen algorithms (in your report, not here).

### 2(b) Qualitative Evaluation [20%]
Visualize the obtained clusters using [Gephi](https://gephi.org) or any other graph visualization tool. Report the visualizations and comment your observations.

In [ ]:
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
from networkx.algorithms.community import louvain_communities
import infomap
import matplotlib.pyplot as plt

def spectral_partition(G, n_groups):
    L = nx.laplacian_matrix(G).astype(float)
    _, vecs = eigsh(L, k=n_groups, which='SM')
    pred = KMeans(n_clusters=n_groups, n_init=10, random_state=0).fit_predict(vecs)
    return dict(zip(G.nodes(), pred))

def infomap_partition(G):
    node_list = list(G.nodes())
    node_to_idx = {n: i for i, n in enumerate(node_list)}

    im = infomap.Infomap(silent=True, two_level=True)
    for u, v in G.edges():
        im.add_link(node_to_idx[u], node_to_idx[v])
    im.run()

    partition = {}
    for node in im.tree:
        if node.is_leaf:
            partition[node_list[node.node_id]] = node.module_id
    return partition

def louvain_partition(G):
    return {n: ci for ci, comm in enumerate(louvain_communities(G)) for n in comm}

graph_collection = {}

for fname, g in classic_graphs.items():
    tag = fname.split('.')[0]
    gt = nx.get_node_attributes(g, 'label')
    k = len(set(gt.values())) if gt else 3
    graph_collection[tag] = {'G': g, 'gt': gt or None, 'k': max(k, 2)}

graph_collection['citeseer'] = {'G': G_citeseer, 'gt': citeseer_labels, 'k': len(set(citeseer_labels.values()))}
graph_collection['cora'] = {'G': G_cora, 'gt': cora_labels, 'k': len(set(cora_labels.values()))}
graph_collection['pubmed'] = {'G': G_pubmed, 'gt': pubmed_labels, 'k': len(set(pubmed_labels.values()))}

methods = [
    ('Spectral', lambda g, k: spectral_partition(
        g if nx.is_connected(g) else g.subgraph(max(nx.connected_components(g), key=len)).copy(), k)),
    ('Infomap', lambda g, k: infomap_partition(g)),
    ('Louvain', lambda g, k: louvain_partition(g)),
]
cluster_output = {}

os.makedirs('gephi_export', exist_ok=True)

for ds, info in graph_collection.items():
    G = info['G']
    n = G.number_of_nodes()
    cluster_output[ds] = {}
    pos = nx.spring_layout(G, seed=42)

    print(f"\n{'='*55}\n{ds}: {n} nodes, {G.number_of_edges()} edges\n")

    for mname, mfunc in methods:
        part = mfunc(G, info['k'])
        part = {nd: part.get(nd, -1) for nd in G.nodes()}
        cluster_output[ds][mname] = part
        fig, ax = plt.subplots(figsize=(10, 8))
        node_cols = [part[nd] for nd in G.nodes()]
        nx.draw(G, pos, node_color=node_cols, cmap=plt.cm.Set3,
                node_size=max(5, 300 // (1 + n // 50)),
                with_labels=n < 50, font_size=5,
                edge_color='#cccccc', width=0.3, ax=ax)
        ax.set_title(f'{ds} — {mname}')
        plt.tight_layout()
        plt.show()
        Gx = nx.Graph()
        Gx.add_nodes_from(G.nodes())
        Gx.add_edges_from(G.edges())
        deg = dict(G.degree())
        deg_max = max(deg.values(), default=1)
        for nd in Gx.nodes():
            Gx.nodes[nd]['cluster'] = int(part.get(nd, -1))
            Gx.nodes[nd]['size'] = round(5 + 20 * deg[nd] / deg_max, 2)

        gml_path = f"gephi_export/{ds}_{mname.lower().replace(' ', '_').replace('.', '')}.gml"
        nx.write_gml(Gx, gml_path)
        print(f'  {mname} -> {gml_path}')

### 2(c) Quantitative Evaluation [20%]
Evaluate the quality of the clusters using label independent (topology only) metrics for both sets of graphs and label dependent metrics for graphs with labels.

*   \tipo{topology based metrics: Modularity and Conductance}
*   \tipo{label dependent metrics: NMI and ARI (report for all datasets that have labels, this should be all but **strike**)}

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

def modularity_score(G, partition):
    groups = defaultdict(set)
    for nd, cid in partition.items():
        groups[cid].add(nd)
    return nx.community.modularity(G, groups.values())

def avg_conductance(G, partition):
    groups = defaultdict(set)
    for nd, cid in partition.items():
        groups[cid].add(nd)

    total_degree = sum(deg for _, deg in G.degree())
    vals = []
    for S in groups.values():
        if len(S) in (0, G.number_of_nodes()):
            continue
        cut = nx.cut_size(G, S)
        vol = sum(deg for _, deg in G.degree(S))
        vol_bar = total_degree - vol
        if min(vol, vol_bar) > 0:
            vals.append(cut / min(vol, vol_bar))
    return float(np.mean(vals)) if vals else 0.0
eval_rows = []

for ds, info in graph_collection.items():
    G = info['G']
    gt = info['gt']

    for mname, partition in cluster_output[ds].items():
        mod = modularity_score(G, partition)
        cond = avg_conductance(G, partition)

        row = {'Dataset': ds, 'Algorithm': mname,
               'Modularity': round(mod, 4), 'Conductance': round(cond, 4),
               'NMI': '—', 'ARI': '—'}
        if gt:
            shared = [n for n in partition if n in gt]
            if shared:
                y_pred = [partition[n] for n in shared]
                y_true = [gt[n] for n in shared]
                row['NMI'] = round(normalized_mutual_info_score(y_true, y_pred), 4)
                row['ARI'] = round(adjusted_rand_score(y_true, y_pred), 4)

        eval_rows.append(row)

eval_df = pd.DataFrame(eval_rows)
print(eval_df.to_string(index=False))

## 3. Evaluation using synthetic datasets: [30%]
1. Create a set of synthetic dataset using [LFR](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.community.LFR_benchmark_graph.html).  [10%]
    *   The common practice is to sample for varying values of $\mu$ which controls how well separated are the communities, i.e. generating synthetic graphs with $\mu=.1$ to $\mu=.9$, reporting average performance for 10 realizations at each difficulty level (90 total), see https://arxiv.org/abs/0805.4770, Fig 5 for example. N = 1000, or 5000 are common settings. For this experiments, you can use $\mu=.5$, n=1000, tau1 = 3, tau2 = 1.5, average degree=5, min community=20.
2. Evaluate the chosen algorithms (quantitatively using ARI/NMI) in the previous questions on this synthetic datasets and report your results [10%]
3. Compute the Average Modularity and Conductance measures for each of the following sets of datasets (i) Real-Classic, (ii) Real-Node-Label and (iii) Synthetic datasets, compare them and report your observations

In [ ]:
mu_range = np.round(np.linspace(0.1, 0.9, 9), 2)
n_realizations = 10

lfr_graphs = []
for mu in mu_range:
    for r in range(n_realizations):
        try:
            G_lfr = nx.generators.community.LFR_benchmark_graph(
                n=1000, tau1=3, tau2=1.5, mu=mu,
                average_degree=5, max_degree=100,
                min_community=20, max_community=100,
                seed=r * 100 + int(mu * 100),
                tol=1e-3, max_iters=1000
            )
            gt = {nd: min(G_lfr.nodes[nd]['community']) for nd in G_lfr.nodes()}
            lfr_graphs.append((mu, r, G_lfr, gt))
            print(f"  LFR mu={mu:.1f} run {r+1}/{n_realizations} OK")
        except Exception as exc:
            print(f"  LFR mu={mu:.1f} run {r+1}/{n_realizations} FAILED: {exc}")

print(f"\nGenerated {len(lfr_graphs)} LFR graphs")
synth_rows = []

for mu, r, G_lfr, gt in lfr_graphs:
    n_comms = max(len(set(gt.values())), 2)
    if nx.is_connected(G_lfr):
        G_conn = G_lfr
    else:
        G_conn = G_lfr.subgraph(max(nx.connected_components(G_lfr), key=len)).copy()

    algo_results = {
        'Spectral': spectral_partition(G_conn, n_comms),
        'Infomap': infomap_partition(G_lfr),
        'Louvain': louvain_partition(G_lfr),
    }

    for aname, part in algo_results.items():
        part = {nd: part.get(nd, -1) for nd in G_lfr.nodes()}

        mod = modularity_score(G_lfr, part)
        cond = avg_conductance(G_lfr, part)

        nodes_with_gt = [n for n in part if n in gt]
        y_pred = [part[n] for n in nodes_with_gt]
        y_true = [gt[n] for n in nodes_with_gt]
        nmi = normalized_mutual_info_score(y_true, y_pred)
        ari = adjusted_rand_score(y_true, y_pred)

        synth_rows.append({
            'mu': mu, 'run': r, 'Algorithm': aname,
            'Modularity': mod, 'Conductance': cond, 'NMI': nmi, 'ARI': ari
        })

    print(f"  Evaluated mu={mu:.1f} run {r+1}")

synth_df = pd.DataFrame(synth_rows)
fig, axes = plt.subplots(2, 2, figsize=(13, 9))
for ax, metric in zip(axes.ravel(), ['NMI', 'ARI', 'Modularity', 'Conductance']):
    for algo in ['Spectral', 'Infomap', 'Louvain']:
        subset = synth_df[synth_df['Algorithm'] == algo]
        grouped = subset.groupby('mu')[metric]
        mu_means = grouped.mean()
        mu_stds = grouped.std()
        ax.errorbar(mu_means.index, mu_means.values, yerr=mu_stds.values,
                    label=algo, marker='o', capsize=4)
    ax.set_xlabel('μ (mixing parameter)')
    ax.set_ylabel(metric)
    ax.set_title(f'{metric} vs μ')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSynthetic Dataset: Mean Performance per μ per Algorithm\n")
pivot = synth_df.groupby(['Algorithm', 'mu'])[['NMI', 'ARI', 'Modularity', 'Conductance']].mean()
print(pivot.round(4).to_string())

print("\nOverall Average across all μ values\n")
overall = synth_df.groupby('Algorithm')[['NMI', 'ARI', 'Modularity', 'Conductance']].mean()
print(overall.round(4).to_string())

In [ ]:
classic_names = {fname.split('.')[0] for fname in classic_graphs.keys()}
labeled_names = {'citeseer', 'cora', 'pubmed'}

rc_mask = eval_df['Dataset'].isin(classic_names)
rl_mask = eval_df['Dataset'].isin(labeled_names)
numeric_cols = ['Modularity', 'Conductance']
avg_rc = eval_df.loc[rc_mask, numeric_cols].apply(pd.to_numeric, errors='coerce').mean()
avg_rl = eval_df.loc[rl_mask, numeric_cols].apply(pd.to_numeric, errors='coerce').mean()
avg_syn = synth_df[numeric_cols].mean()

comparison_df = pd.DataFrame({
    'Real-Classic': avg_rc,
    'Real-Node-Label': avg_rl,
    'Synthetic (LFR)': avg_syn,
}).T

print("Average Modularity & Conductance by Dataset Category (all)\n")
print(comparison_df.round(4).to_string())
print("\n\nPer-Algorithm Breakdown by Category\n")

for algo in ['Spectral', 'Infomap', 'Louvain']:
    rc_algo = eval_df.loc[rc_mask & (eval_df['Algorithm'] == algo), numeric_cols].apply(pd.to_numeric, errors='coerce').mean()
    rl_algo = eval_df.loc[rl_mask & (eval_df['Algorithm'] == algo), numeric_cols].apply(pd.to_numeric, errors='coerce').mean()
    syn_algo = synth_df.loc[synth_df['Algorithm'] == algo, numeric_cols].mean()

    tbl = pd.DataFrame({
        'Real-Classic': rc_algo,
        'Real-Node-Label': rl_algo,
        'Synthetic (LFR)': syn_algo,
    }).T

    print(f"--- {algo} ---")
    print(tbl.round(4).to_string())
    print()

## 4. [Bonus] Compare the results with an algorithm published/proposed in the last 4 years and report your observations [10%]

In [ ]:
import igraph as ig
import leidenalg

def leiden_partition(G):
    """Community detection via the Leiden algorithm."""
    node_list = list(G.nodes())
    node_to_idx = {n: i for i, n in enumerate(node_list)}
    g_ig = ig.Graph(n=len(node_list), directed=False)
    g_ig.add_edges([(node_to_idx[u], node_to_idx[v]) for u, v in G.edges()])
    part = leidenalg.find_partition(g_ig, leidenalg.ModularityVertexPartition)
    return {node_list[i]: m for i, m in enumerate(part.membership)}
print("=== Leiden vs Spectral / Infomap / Louvain on Real-World Datasets ===\n")

leiden_rows = []
for ds, info in graph_collection.items():
    G = info['G']
    gt = info['gt']
    part = leiden_partition(G)
    part = {nd: part.get(nd, -1) for nd in G.nodes()}

    mod = modularity_score(G, part)
    cond = avg_conductance(G, part)
    row = {'Dataset': ds, 'Algorithm': 'Leiden',
           'Modularity': round(mod, 4), 'Conductance': round(cond, 4),
           'NMI': '—', 'ARI': '—'}
    if gt:
        shared = [n for n in part if n in gt]
        if shared:
            y_pred = [part[n] for n in shared]
            y_true = [gt[n] for n in shared]
            row['NMI'] = round(normalized_mutual_info_score(y_true, y_pred), 4)
            row['ARI'] = round(adjusted_rand_score(y_true, y_pred), 4)
    leiden_rows.append(row)

leiden_real_df = pd.DataFrame(leiden_rows)
combined_real = pd.concat([eval_df, leiden_real_df], ignore_index=True)
print(combined_real.to_string(index=False))
print("\n\nLeiden on LFR Synthetic Graphs\n")

leiden_synth_rows = []
for mu, r, G_lfr, gt in lfr_graphs:
    part = leiden_partition(G_lfr)
    part = {nd: part.get(nd, -1) for nd in G_lfr.nodes()}
    mod = modularity_score(G_lfr, part)
    cond = avg_conductance(G_lfr, part)
    nodes_with_gt = [n for n in part if n in gt]
    y_pred = [part[n] for n in nodes_with_gt]
    y_true = [gt[n] for n in nodes_with_gt]
    nmi = normalized_mutual_info_score(y_true, y_pred)
    ari = adjusted_rand_score(y_true, y_pred)
    leiden_synth_rows.append({
        'mu': mu, 'run': r, 'Algorithm': 'Leiden',
        'Modularity': mod, 'Conductance': cond, 'NMI': nmi, 'ARI': ari
    })

leiden_synth_df = pd.DataFrame(leiden_synth_rows)
all_synth_df = pd.concat([synth_df, leiden_synth_df], ignore_index=True)

fig, axes = plt.subplots(2, 2, figsize=(13, 9))
for ax, metric in zip(axes.ravel(), ['NMI', 'ARI', 'Modularity', 'Conductance']):
    for algo in ['Spectral', 'Infomap', 'Louvain', 'Leiden']:
        subset = all_synth_df[all_synth_df['Algorithm'] == algo]
        grouped = subset.groupby('mu')[metric]
        mu_means = grouped.mean()
        mu_stds = grouped.std()
        ax.errorbar(mu_means.index, mu_means.values, yerr=mu_stds.values,
                    label=algo, marker='o', capsize=4)
    ax.set_xlabel('μ (mixing parameter)')
    ax.set_ylabel(metric)
    ax.set_title(f'{metric} vs μ (incl. Leiden)')
    ax.legend()
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print("\nSynthetic: Overall Average (4 algos)\n")
summary = all_synth_df.groupby('Algorithm')[['NMI', 'ARI', 'Modularity', 'Conductance']].mean()
print(summary.round(4).to_string())

## 5. [Bonus] Compare the results on 3 more real world datasets not included in the assignment and report your observations [10%]

In [ ]:
import requests, gzip, io

bonus_graphs = {}
G_lesmis = nx.les_miserables_graph()
bonus_graphs['les_miserables'] = {'G': G_lesmis, 'gt': None,
                                   'k': 6} 
print(f"Les Misérables: {G_lesmis.number_of_nodes()} nodes, {G_lesmis.number_of_edges()} edges (no ground-truth labels)")
try:
    r1 = requests.get("https://snap.stanford.edu/data/email-Eu-core.txt.gz", timeout=30)
    with gzip.open(io.BytesIO(r1.content), 'rt') as f:
        G_email = nx.parse_edgelist(
            [line for line in f if not line.startswith('#')], nodetype=int)

    r2 = requests.get("https://snap.stanford.edu/data/email-Eu-core-department-labels.txt.gz", timeout=30)
    email_labels = {}
    with gzip.open(io.BytesIO(r2.content), 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue
            parts = line.strip().split()
            if len(parts) == 2:
                email_labels[int(parts[0])] = int(parts[1])
    for n, l in email_labels.items():
        if n in G_email:
            G_email.nodes[n]['label'] = l
    bonus_graphs['email_eu_core'] = {
        'G': G_email, 'gt': email_labels,
        'k': len(set(email_labels.values()))}
    print(f"Email-Eu-core: {G_email.number_of_nodes()} nodes, {G_email.number_of_edges()} edges, "
          f"{len(set(email_labels.values()))} departments")
except Exception as e:
    print(f"Email-Eu-core download failed: {e}")

try:
    r3 = requests.get("https://snap.stanford.edu/data/facebook_combined.txt.gz", timeout=30)
    with gzip.open(io.BytesIO(r3.content), 'rt') as f:
        G_fb = nx.parse_edgelist(
            [line for line in f if not line.startswith('#')], nodetype=int)
    bonus_graphs['facebook'] = {'G': G_fb, 'gt': None, 'k': 10}
    print(f"Facebook: {G_fb.number_of_nodes()} nodes, {G_fb.number_of_edges()} edges (no ground-truth labels)")
except Exception as e:
    print(f"Facebook download failed: {e}")

all_algos = [
    ('Spectral', lambda g, k: spectral_partition(
        g if nx.is_connected(g) else g.subgraph(max(nx.connected_components(g), key=len)).copy(), k)),
    ('Infomap', lambda g, k: infomap_partition(g)),
    ('Louvain', lambda g, k: louvain_partition(g)),
    ('Leiden', lambda g, k: leiden_partition(g)),
]

bonus_rows = []
for ds, info in bonus_graphs.items():
    G = info['G']
    gt = info['gt']
    print(f"\nRunning algorithms on {ds} ...")

    for aname, afunc in all_algos:
        part = afunc(G, info['k'])
        part = {nd: part.get(nd, -1) for nd in G.nodes()}

        mod = modularity_score(G, part)
        cond = avg_conductance(G, part)
        row = {'Dataset': ds, 'Algorithm': aname,
               'Modularity': round(mod, 4), 'Conductance': round(cond, 4),
               'NMI': '—', 'ARI': '—'}
        if gt:
            shared = [n for n in part if n in gt]
            if shared:
                y_pred = [part[n] for n in shared]
                y_true = [gt[n] for n in shared]
                row['NMI'] = round(normalized_mutual_info_score(y_true, y_pred), 4)
                row['ARI'] = round(adjusted_rand_score(y_true, y_pred), 4)
        bonus_rows.append(row)
        print(f"  {aname} done")

bonus_df = pd.DataFrame(bonus_rows)
print("\n=== Bonus Datasets: Full Evaluation ===\n")
print(bonus_df.to_string(index=False))